In [1]:
!pip install numpy

import numpy as np

You should consider upgrading via the '/home/conn/anaconda3/envs/BanditWorkshop2/bin/python -m pip install --upgrade pip' command.


## Bandit Environment
### Call Bandit(min, max) to create a bandit with range or rewards from min to max
### Call Bandit.sample() to sample from this distribution
### sample() will return a random value in the reward range

In [2]:
#Bandit instance, used as environment
class Bandit:
    def __init__(self, min_val, max_val):
        self.times_chosen = 1
        self.dist_range = sorted(np.random.randint(min_val, max_val+1, 2))
        self.true_reward = (self.dist_range[0] + self.dist_range[1]) / 2 # Mean of the reward distribution
        print(self.dist_range)

    def sample(self):
        self.times_chosen += 1
        # Return a random value from the reward distribution (uniform between dist_range[0] and dist_range[1])
        #TUTORIAL: how can we sample this random value? (hint. np.random.uniform function might be a good start)
        return np.random.uniform(self.dist_range[0], self.dist_range[1])

## Our Agent needs to:
###    a) Keep track of how "good" each bandit has been in terms of reward
###    b) Keep track of regret

In [3]:
class Agent:
    def __init__(self, num_bandits=10, dist_min=1, dist_max=10):
        self.num_bandits = num_bandits
        self.bandits = []
        self.rewards_avg = [] # What we assume the reward to be, based on what we have seen. Will constantly update as we sample more
        self.best_reward = -1
        self.regret = 0
        self.time = 1
        #Create bandits for the agent to interact with
        for i in range(self.num_bandits):
            self.bandits.append(Bandit(dist_min, dist_max))
            self.rewards_avg.append(1)
            #We want to keep track of what's the best possible reward (on average)
            #TUTORIAL: how can we keep track of this best reward
            if self.bandits[i].true_reward > self.best_reward:
                self.best_reward = self.bandits[i].true_reward

# Greedy Agent
## The Greedy Agent chooses the best action at every time (pure exploitation). Is this what we want? What are the advantages and disadvantages of this?
## action = $arg max(\text{Bandit rewards})$

In [4]:
class GreedyAgent(Agent):
    def __init__(self):
        super().__init__()

    def choose_bandit(self):
        #Choose best action
        #TUTORIAL: how do we choose the index of the highest reward bandit? (hint: we have some variables in the Agent superclass that keep track of every bandit's presumed values)
        bandit_idx = np.argmax(self.rewards_avg)
        reward = self.bandits[bandit_idx].sample()

        #Calculate new reward average
        new_reward_avg = ((self.rewards_avg[bandit_idx] * (self.bandits[bandit_idx].times_chosen-1)) + reward) / self.bandits[bandit_idx].times_chosen
        self.rewards_avg[bandit_idx] = new_reward_avg
        #Calculate regret
        self.regret += (self.best_reward - self.bandits[bandit_idx].true_reward)
        print("GREEDY: Bandit {} was chosen ({}, {}), with reward {}, and regret {}, best reward {}".format(bandit_idx, self.bandits[bandit_idx].dist_range[0], self.bandits[bandit_idx].dist_range[1], reward, self.best_reward - self.bandits[bandit_idx].true_reward, self.best_reward))

# Optimistic Greedy Agent
## This agent works the same as the greedy agent, except the expected reward is set high for all bandits initially, so that the mean value is brought "down", which means that one agent shouldn't ever be always selected after the first run

In [5]:
class OptimisticGreedyAgent(Agent):
    def __init__(self):
        super().__init__()
        for i in range(len(self.rewards_avg)):
            #TUTORIAL: What value can we set the rewards_avg to?
            self.rewards_avg[i] = 10

    def choose_bandit(self):
        #Choose best action
        #TUTORIAL (same as greedy agent)
        bandit_idx = np.argmax(self.rewards_avg)
        reward = self.bandits[bandit_idx].sample()

        #Calculate new reward average
        new_reward_avg = ((self.rewards_avg[bandit_idx] * (self.bandits[bandit_idx].times_chosen-1)) + reward) / self.bandits[bandit_idx].times_chosen
        self.rewards_avg[bandit_idx] = new_reward_avg
        #Calculate regret
        self.regret += (self.best_reward - self.bandits[bandit_idx].true_reward)
        print("OPTIMISTIC GREEDY: Bandit {} was chosen ({}, {}), with reward {}, and regret {}, best reward {}".format(bandit_idx, self.bandits[bandit_idx].dist_range[0], self.bandits[bandit_idx].dist_range[1], reward, self.best_reward - self.bandits[bandit_idx].true_reward, self.best_reward))

# e-Greedy Agent
## The Epsilon Greedy Agent chooses the best action most of the time (exploitation) but every so often chooses a random action (exploration). Why might this work? What are the advantages and disadvantages of this?
## action = USUALLY $argmax(\text{Bandit rewards})$ but SOMETIMES $random(Bandit)$

In [6]:
class EpsilonGreedyAgent(Agent):
    def __init__(self, epsilon = 0.9):
        super().__init__()
        self.epsilon = epsilon

    def choose_bandit(self):
        if np.random.uniform() > self.epsilon: # Choose random action
            bandit_idx = np.random.randint(0, self.num_bandits)
        else: # Choose best action
            bandit_idx = np.argmax(self.rewards_avg)
        reward = self.bandits[bandit_idx].sample()
        #Calculate new reward average
        new_reward_avg = ((self.rewards_avg[bandit_idx] * (self.bandits[bandit_idx].times_chosen-1)) + reward) / self.bandits[bandit_idx].times_chosen
        self.rewards_avg[bandit_idx] = new_reward_avg
        #Calculate regret
        self.regret += (self.best_reward - self.bandits[bandit_idx].true_reward)
        print("EPSILON GREEDY: Bandit {} was chosen ({}, {}), with reward {}, and regret {}, best reward {}".format(bandit_idx, self.bandits[bandit_idx].dist_range[0], self.bandits[bandit_idx].dist_range[1], reward, self.best_reward - self.bandits[bandit_idx].true_reward, self.best_reward))

# UCB Agent
## The Upper Confidence Bound Agent chooses the agent that has the maximum best reward + least confidence. This means that actions chosen long ago are more uncertain on their reward than actions chosen recently. We assign a "confidence" to these actions (based on our confidence on how good/bad they actually were), and then choose our action based on the reward we think they give, plus the confidence
## action = $arg max [\text{bandit reward} + confidence * \sqrt{\frac{log(t)}{ N_t a}} ]$
### Where $t$ is total time and $N_t$ is number of times the action has been selected

In [7]:
class UCBAgent(Agent):
    def __init__(self, confidence=2):
        super().__init__()
        self.confidence = confidence
        self.times_since_last_choice = [1 for i in range(self.num_bandits)]

    def choose_bandit(self):
        confidences = []
        for bandit_idx in range(len(self.bandits)):
            est_reward = self.rewards_avg[bandit_idx]
            #TUTORIAL: Calculate the confidence value. Each agent has a .time value for how many actions have been taken, and each bandit has a .times_chosen
            confidence_val = self.confidence * np.sqrt(np.log(self.time)/self.bandits[bandit_idx].times_chosen)
            confidences.append(est_reward + confidence_val)
        self.time += 1
        bandit_idx = np.argmax(confidences)
        reward = self.bandits[bandit_idx].sample()
        #Calculate new reward average
        old_rewards_avg = self.rewards_avg[bandit_idx]
        new_reward_avg = ((self.rewards_avg[bandit_idx] * (self.bandits[bandit_idx].times_chosen-1)) + reward) / self.bandits[bandit_idx].times_chosen
        self.rewards_avg[bandit_idx] = new_reward_avg
        #Calculate regret
        self.regret += (self.best_reward - self.bandits[bandit_idx].true_reward)
        print("UCB: Bandit {} was chosen ({}, {}), with reward {}, expected_reward {}, confidence {}, and regret {}, best reward {}".format(bandit_idx, self.bandits[bandit_idx].dist_range[0], self.bandits[bandit_idx].dist_range[1], reward, old_rewards_avg, confidences[bandit_idx], self.best_reward - self.bandits[bandit_idx].true_reward, self.best_reward))


In [11]:
if __name__ == "__main__":
    agent = UCBAgent()
    for i in range(100):
        agent.choose_bandit()
    print("Final regret: {}".format(agent.regret))

[6, 9]
[2, 8]
[4, 8]
[9, 9]
[3, 7]
[2, 9]
[3, 8]
[4, 8]
[8, 10]
[1, 4]
UCB: Bandit 0 was chosen (6, 9), with reward 8.049650481247589, expected_reward 1, confidence 1.0, and regret 1.5, best reward 9.0
UCB: Bandit 0 was chosen (6, 9), with reward 7.876798105561187, expected_reward 4.5248252406237945, confidence 5.702235263139269, and regret 1.5, best reward 9.0
UCB: Bandit 0 was chosen (6, 9), with reward 8.957896702951283, expected_reward 5.642149528936258, confidence 6.852445519547982, and regret 1.5, best reward 9.0
UCB: Bandit 0 was chosen (6, 9), with reward 7.428815405468189, expected_reward 6.471086322440014, confidence 7.648496344955489, and regret 1.5, best reward 9.0
UCB: Bandit 0 was chosen (6, 9), with reward 7.104719396690154, expected_reward 6.66263213904565, confidence 7.7973348886445395, and regret 1.5, best reward 9.0
UCB: Bandit 0 was chosen (6, 9), with reward 8.7924414469997, expected_reward 6.736313348653067, confidence 7.829248073519426, and regret 1.5, best rewar